In [24]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import random
import os
from collections import defaultdict
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras import layers, optimizers, backend, models
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from IPython.display import display
import argparse
import tensorflow as tf

In [25]:
def show_samples(data_path, img_path):
	all_imgs = os.listdir(img_path)
	sample_img = random.sample(all_imgs, 9)
	sample_img_path   =  [  os.path.join(img_path, sample)  for sample in sample_img ]
	
	fig, axes = plt.subplots(3, 3 , figsize = (10, 10) )
	for ax, img in zip(axes.flatten(), sample_img_path):
		image = Image.open(img)
		ax.imshow(image)
		ax.axis('off')
	
	plt.tight_layout()
	plt.subplots_adjust(top = 0.9)
	plt.suptitle("Sample Images" , fontsize = 16)
	plt.show()


In [26]:
persimmon_model  = load_model('/kaggle/input/persimmon_bansi_inceptionv3/keras/default/1/cabbage_model.h5')


In [27]:
patterns = [ "11_1TOP", "2_2FR45", "7_2FR45", "12_2FR45", "17_2FR45",
        "22_2FR45", "27_2FR45", "32_2FR45", "37_2FR45", "21_1TOP" ]


def select_imgs(img_path):
    selected_imgs_list = [img for img in img_path if any(pattern in img for pattern in patterns) ]
    return selected_imgs_list


data_path_2 = '/kaggle/input/test-persimmon-bansi'

img_path_2s = os.path.join(data_path_2, 'persimmon_bansi_S')
img_path_2m = os.path.join(data_path_2, 'persimmon_bansi_M')
img_path_2l = os.path.join(data_path_2, 'persimmon_bansi_L')

img_2s =  os.listdir(img_path_2s)
img_2m =  os.listdir(img_path_2m)
img_2l =  os.listdir(img_path_2l)

img_2s_list = select_imgs(img_2s)
img_2m_list = select_imgs(img_2m)
img_2l_list = select_imgs(img_2l)


test_img_s =  [os.path.join(img_path_2s, img) for img  in img_2s_list if "76" not in img]
test_img_m =  [os.path.join(img_path_2m, img) for img  in img_2m_list if "76" not in img]
test_img_l =  [os.path.join(img_path_2l, img) for img  in img_2l_list if "76" not in img]



test_persimmon_bansi_s = sorted(test_img_s, key = lambda x: ( \
                                    int(x.split("bansi_S_")[1].split("-")[0]),
                                    int(x.split("bansi_S_")[1].split("-")[1].split("_")[0])
                                   ) )

test_persimmon_bansi_m = sorted(test_img_m, key = lambda x: ( \
                                    int(x.split("bansi_M_")[1].split("-")[0]),
                                    int(x.split("bansi_M_")[1].split("-")[1].split("_")[0])
                                   ) )

test_persimmon_bansi_l = sorted(test_img_l, key = lambda x: ( \
                                    int(x.split("bansi_L_")[1].split("-")[0]),
                                    int(x.split("bansi_L_")[1].split("-")[1].split("_")[0])
                                   ) )

test_images = []
test_labels = []


def test_load_and_label(image_path, label):
    for image in image_path:
        img = load_img(image, target_size = (224, 224))
        img_array =img_to_array(img) / 225.0
        test_images.append(img_array)
        test_labels.append(label)

test_load_and_label(test_persimmon_bansi_s, 0)
test_load_and_label(test_persimmon_bansi_m, 1)
test_load_and_label(test_persimmon_bansi_l, 2)


test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [28]:
test_persimmon_bansi_l[:5]

['/kaggle/input/test-persimmon-bansi/persimmon_bansi_L/persimmon_bansi_L_77-2_2FR45.png',
 '/kaggle/input/test-persimmon-bansi/persimmon_bansi_L/persimmon_bansi_L_77-7_2FR45.png',
 '/kaggle/input/test-persimmon-bansi/persimmon_bansi_L/persimmon_bansi_L_77-11_1TOP.png',
 '/kaggle/input/test-persimmon-bansi/persimmon_bansi_L/persimmon_bansi_L_77-12_2FR45.png',
 '/kaggle/input/test-persimmon-bansi/persimmon_bansi_L/persimmon_bansi_L_77-17_2FR45.png']

In [30]:
grouped_images = defaultdict(list)


test_img_paths = test_persimmon_bansi_s + test_persimmon_bansi_m + test_persimmon_bansi_l


for img_path in test_img_paths:
	fruit_id = img_path.split("-")[2].split("persimmon_bansi_")[2]
	grouped_images[fruit_id].append(img_path)

def predict_group(model, image_paths):
	preds = []

	for img_path in image_paths:
		img = load_img(img_path, target_size = (224, 224))
		img_array = img_to_array(img) / 255.0
		img_array = np.expand_dims(img_array, axis = 0)

		pred = model.predict(img_array)
		preds.append(pred)

	avg_pred = np.mean(preds, axis = 0)
	
	final_class = np.argmax(avg_pred)

	return final_class, avg_pred
    

In [31]:
correct_predictions = 0
total_predictions = 0

for fruit_id, img_paths in grouped_images.items():
    if 'S' in fruit_id:
        true_label = 0
    elif 'M' in img_paths[0]:
        true_label = 1
    elif 'L' in img_paths[0]:
        true_label = 2
    else:
        continue
    
    final_class, avg_pred = predict_group(persimmon_model, img_paths)

    if final_class == true_label:
        correct_predictions += 1

    total_predictions += 1

accuracy = correct_predictions/total_predictions

print(accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━

In [ ]:


images = []
labels = []
test_images = []
test_labels = []

def load_and_label(image_path, label):
    for image in image_path:
        img = load_img(image, target_size = (224, 224))
        img_array =img_to_array(img) / 225.0
        images.append(img_array)
        labels.append(label)


load_and_label(training_persimmon_bansi_s , 0)
load_and_label(training_persimmon_bansi_m, 1)
load_and_label(training_persimmon_bansi_l, 2)

def test_load_and_label(image_path, label):
    for image in image_path:
        img = load_img(image, target_size = (224, 224))
        img_array =img_to_array(img) / 225.0
        test_images.append(img_array)
        test_labels.append(label)

test_load_and_label(test_persimmon_bansi_s, 0)
test_load_and_label(test_persimmon_bansi_m, 1)
test_load_and_label(test_persimmon_bansi_l, 2)

images = np.array(images)
labels = np.array(labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size = 0.2, random_state = 42)


print(X_train.shape[0])
print(X_val.shape[0])
print(images.shape)

In [ ]:
train_datagen = ImageDataGenerator(
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, y_train, batch_size = 32)
val_generator = val_datagen.flow(X_val, y_val, batch_size = 32)
test_generator = test_datagen.flow(test_images, test_labels, batch_size = 32)

In [ ]:



backend.clear_session()
model_base = InceptionV3(weights = 'imagenet', include_top = False )

InceptionV3_model = model_base.output
pool = GlobalAveragePooling2D()(InceptionV3_model)
dense_1 = layers.Dense(512, activation = 'relu')(pool)
output = layers.Dense(3, activation = 'softmax')(dense_1)


model_InceptionV3 = Model(inputs = model_base.input, outputs = output)
model_InceptionV3.compile(loss = 'sparse_categorical_crossentropy', 
                          optimizer = optimizers.SGD(learning_rate = 1e-4, momentum = 0.9),
                          metrics = ['accuracy']
                         )


In [ ]:

history = model_InceptionV3.fit(
    train_generator, 
    epochs = 25,
    validation_data = val_generator,
    verbose = 1,
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience= 5, restore_best_weights = True)]
)

loss, accuracy= model_InceptionV3.evaluate(test_generator)
print(loss, accuracy)